<a href="https://colab.research.google.com/github/Mohibl-6568/Clothing-Recommender-System/blob/main/ClothingRecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time
import math

In [10]:
# Load and Preprocess Data
df = pd.read_csv('Clothify-fyp-data - main.csv')
df = df[df['Title'].notna() & df['Title'].str.strip().ne('')]
df = df.drop('Owner', axis=1)

df.head()

,ID,Title,"For (Male, Female, both)",Type 1,Type 2,Tag1,Tag2,Tag3,Tag4,Tag5,Tag6,Tag7,Tag8,Tag9,Tag10,Tag11,Tag12,Tag13,Tag14,Tag15
0,1,Printed hoodies,both,Top,Hoodie,Printed hoodie,Unisex,pullover hoodie,pocket hoodie,kangaroo font,white hoodie,black hoodie,royal blue hoodie,gray hoodie,navy hoodie,washable,stylish,custom design,winter collection,Astronout design
1,2,Australian Map hoodie,both,Top,Hoodie,Printed hoodie,Unisex,pullover hoodie,pocket hoodie,kangaroo font,white hoodie,black hoodie,royal blue hoodie,gray hoodie,navy hoodie,washable,stylish,custom design,winter collection,Australia map hoodie
2,3,Devil Tank Top,Male,Top,Tank,Printed Tank Top,Devilish design,Summer collection,Black Tank,White Tank,Gray Tank,Breathable,Cotton,Sleeveless,Stylish,Custom made,washable,comfy,sleepwear,unique design
3,4,Floral print summer dress,Female,Dress,Outfit,Floral printed dress,Summer dress,Summer collection,Floral design,Floral print,flowers,custom design,breathable,flowy,short sleeve,round neck,above knee,casual dress,T-shirt style dress,milk-silk fabric
4,5,Printed T-shirt,Both,Top,T-shirt,Black T-shirt,Cotton,Summer collection,black tee,summer wear,printed Tee,breathable,casual tee,round neck,unisex,NaN,NaN,NaN,NaN,NaN


In [11]:
# Standardize gender labels
df['For (Male, Female, both)'] = df['For (Male, Female, both)'].replace({
    'male': 'Male', 'female': 'Female', 'both': 'Both', 'unisex': 'Both'
})

In [12]:
# Define tag columns
tag_columns = [col for col in df.columns if 'Tag' in col]

# Combine tags into a single string column
def combine_tags(df, tag_columns):
    return df[tag_columns].fillna('').apply(lambda row: ' '.join(map(str, row)), axis=1)

df['combined_tags'] = combine_tags(df, tag_columns).str.lower()

In [13]:
male_df = df[df['For (Male, Female, both)'].isin(['Male', 'Both'])].copy()
female_df = df[df['For (Male, Female, both)'].isin(['Female', 'Both'])].copy()

In [14]:
male_df.head()

,ID,Title,"For (Male, Female, both)",Type 1,Type 2,Tag1,Tag2,Tag3,Tag4,Tag5,...,Tag7,Tag8,Tag9,Tag10,Tag11,Tag12,Tag13,Tag14,Tag15,combined_tags
0,1,Printed hoodies,Both,Top,Hoodie,Printed hoodie,Unisex,pullover hoodie,pocket hoodie,kangaroo font,...,black hoodie,royal blue hoodie,gray hoodie,navy hoodie,washable,stylish,custom design,winter collection,Astronout design,printed hoodie unisex pullover hoodie pocket h...
1,2,Australian Map hoodie,Both,Top,Hoodie,Printed hoodie,Unisex,pullover hoodie,pocket hoodie,kangaroo font,...,black hoodie,royal blue hoodie,gray hoodie,navy hoodie,washable,stylish,custom design,winter collection,Australia map hoodie,printed hoodie unisex pullover hoodie pocket h...
2,3,Devil Tank Top,Male,Top,Tank,Printed Tank Top,Devilish design,Summer collection,Black Tank,White Tank,...,Breathable,Cotton,Sleeveless,Stylish,Custom made,washable,comfy,sleepwear,unique design,printed tank top devilish design summer collec...
4,5,Printed T-shirt,Both,Top,T-shirt,Black T-shirt,Cotton,Summer collection,black tee,summer wear,...,breathable,casual tee,round neck,unisex,NaN,NaN,NaN,NaN,NaN,black t-shirt cotton summer collection black t...
5,6,Dragon Design Sweatpants,Both,Bottom,Gymwear,Dragon design sweatpants,Cotton,Sweats,Trousers,White Trousers,...,breathable,Comfy,Gym collection,skin friendly,sweat abosrbing,pocket sweats,washable,unisex,NaN,dragon design sweatpants cotton sweats trouser...


In [15]:
female_df.head()

,ID,Title,"For (Male, Female, both)",Type 1,Type 2,Tag1,Tag2,Tag3,Tag4,Tag5,...,Tag7,Tag8,Tag9,Tag10,Tag11,Tag12,Tag13,Tag14,Tag15,combined_tags
0,1,Printed hoodies,Both,Top,Hoodie,Printed hoodie,Unisex,pullover hoodie,pocket hoodie,kangaroo font,...,black hoodie,royal blue hoodie,gray hoodie,navy hoodie,washable,stylish,custom design,winter collection,Astronout design,printed hoodie unisex pullover hoodie pocket h...
1,2,Australian Map hoodie,Both,Top,Hoodie,Printed hoodie,Unisex,pullover hoodie,pocket hoodie,kangaroo font,...,black hoodie,royal blue hoodie,gray hoodie,navy hoodie,washable,stylish,custom design,winter collection,Australia map hoodie,printed hoodie unisex pullover hoodie pocket h...
3,4,Floral print summer dress,Female,Dress,Outfit,Floral printed dress,Summer dress,Summer collection,Floral design,Floral print,...,custom design,breathable,flowy,short sleeve,round neck,above knee,casual dress,T-shirt style dress,milk-silk fabric,floral printed dress summer dress summer colle...
4,5,Printed T-shirt,Both,Top,T-shirt,Black T-shirt,Cotton,Summer collection,black tee,summer wear,...,breathable,casual tee,round neck,unisex,NaN,NaN,NaN,NaN,NaN,black t-shirt cotton summer collection black t...
5,6,Dragon Design Sweatpants,Both,Bottom,Gymwear,Dragon design sweatpants,Cotton,Sweats,Trousers,White Trousers,...,breathable,Comfy,Gym collection,skin friendly,sweat abosrbing,pocket sweats,washable,unisex,NaN,dragon design sweatpants cotton sweats trouser...


In [16]:
# Login + Gender-Based Filtering
print("Welcome to Clothify Interactive Recommender!")
username = input("👤 Enter your username: ").strip()
gender = input("Enter your gender (Male/Female): ").strip().capitalize()

if gender not in ['Male', 'Female']:
    print("Invalid gender input. Showing unfiltered recommendations.")
    filtered_df = df.copy()
else:
    if gender == 'Male':
        filtered_df = male_df
    else:
        filtered_df = female_df

filtered_df['combined_tags'] = combine_tags(filtered_df, tag_columns).str.lower()

Welcome to Clothify Interactive Recommender!
👤 Enter your username: Mohib
Enter your gender (Male/Female): Male


In [17]:
# TF-IDF Tag Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(filtered_df['combined_tags'])

In [18]:
# Interactive Recommendation Logic with Time Decay
# Format: { tag: [(weight, timestamp), ...] }
user_tag_history = defaultdict(list)
seen_items = set()
DECAY_LAMBDA = 0.01  # Decay rate

def current_time():
    return time.time()

def build_user_vector(tag_history, vectorizer):
    if not tag_history:
        return None
    now = current_time()
    weighted_tag_list = []

    for tag, entries in tag_history.items():
        total_weight = 0
        for weight, timestamp in entries:
            age = now - timestamp
            decayed_weight = weight * math.exp(-DECAY_LAMBDA * age)
            total_weight += decayed_weight
        weighted_tag_list.extend([tag] * int(round(total_weight)))

    if not weighted_tag_list:
        return None

    weighted_text = ' '.join(weighted_tag_list)
    return vectorizer.transform([weighted_text])

def recommend_items(user_vector, tfidf_matrix, seen_items, filtered_df, top_n=5):
    if user_vector is None:
        unseen_indices = list(set(filtered_df.index) - seen_items)
        return unseen_indices[:top_n]  # fallback
    similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()
    recommendations = [(i, sim) for i, sim in zip(filtered_df.index, similarities) if i not in seen_items]
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return [i for i, _ in recommendations[:top_n]]

In [20]:
# Recommendation Loop
print(f"\nHello, {username}! Let's get started with your personalized recommendations.\n")

for round_num in range(3):  # Number of recommendation rounds
    print(f"\nRound {round_num + 1}")
    user_vector = build_user_vector(user_tag_history, vectorizer)
    candidate_indices = recommend_items(user_vector, tfidf_matrix, seen_items, filtered_df)

    for idx in candidate_indices:
        item = filtered_df.loc[idx]
        print(f"\n Item {idx}: {item['Title']}")
        print("Tags:", item['combined_tags'])
        feedback = input("Do you like this item? (L = Like / D = Dislike): ").strip().upper()
        seen_items.add(idx)

        tags = item['combined_tags'].split()
        timestamp = current_time()

        if feedback == 'L':
            for tag in tags:
                user_tag_history[tag].append((1, timestamp))
        elif feedback == 'D':
            for tag in tags:
                user_tag_history[tag].append((-1, timestamp))
        else:
            print("Invalid input. Skipping this item.")

    print("\nCurrent tag interaction history (latest):")
    for tag, history in list(user_tag_history.items())[:5]:  # show top 5
        print(f"{tag}: {len(history)} interactions")
    print("Seen items:", len(seen_items))

print("\nThank you for using Clothify! Come back soon for more recommendations.")


Hello, Mohib! Let's get started with your personalized recommendations.


Round 1

 Item 0: Printed hoodies
Tags: printed hoodie unisex pullover hoodie pocket hoodie kangaroo font white hoodie black hoodie royal blue hoodie gray hoodie navy hoodie washable stylish custom design winter collection astronout design
Do you like this item? (L = Like / D = Dislike): L

 Item 1: Australian Map hoodie
Tags: printed hoodie unisex pullover hoodie pocket hoodie kangaroo font white hoodie black hoodie royal blue hoodie gray hoodie navy hoodie washable stylish custom design winter collection australia map hoodie
Do you like this item? (L = Like / D = Dislike): D

 Item 2: Devil Tank Top
Tags: printed tank top devilish design summer collection black tank white tank gray tank breathable cotton sleeveless stylish custom made washable comfy sleepwear unique design
Do you like this item? (L = Like / D = Dislike): L

 Item 4: Printed T-shirt
Tags: black t-shirt cotton summer collection black tee summer 